# Using RDF Classify

## Setup 

In [10]:
import json
import random
import rdflib

import pandas as pd
from datetime import datetime
from zipfile import ZipFile

## Extracting RDF Entity Graphs

The `extract_graph` function takes the path to the exported Sinopia entities zip file, creates a graph and parses the JSON-LD of each entity into the graph and then returns the populated graph.

In [11]:
def extract_graph(zip_filepath):
    graph = rdflib.Graph()

    with ZipFile(zip_filepath) as zip_file:
        for zip_info in zip_file.infolist():
            if zip_info.file_size < 1:
                continue
            with zip_file.open(zip_info) as zip_extract:
                raw_rdf = zip_extract.read()
                try:
                    graph.parse(data=raw_rdf, format='json-ld')
                except json.JSONDecodeError:
                    print(f"Failed to parse {zip_info.filename}")
    return graph

In [12]:
dev_graph = extract_graph("../data/input/sinopia_dev_export_all_2020-01-05T00_01_11.968Z.zip")

Failed to parse sinopia_export_all_2020-01-05T00:01:11.968Z/northwestern_2020-01-05T00:01:13.871Z/complete.log
Failed to parse sinopia_export_all_2020-01-05T00:01:11.968Z/tamu_2020-01-05T00:01:14.022Z/complete.log
Failed to parse sinopia_export_all_2020-01-05T00:01:11.968Z/cornell_2020-01-05T00:01:14.138Z/complete.log
Failed to parse sinopia_export_all_2020-01-05T00:01:11.968Z/hrc_2020-01-05T00:01:12.091Z/complete.log
Failed to parse sinopia_export_all_2020-01-05T00:01:11.968Z/michigan_2020-01-05T00:01:11.994Z/complete.log
Failed to parse sinopia_export_all_2020-01-05T00:01:11.968Z/washington_2020-01-05T00:01:12.787Z/complete.log
Failed to parse sinopia_export_all_2020-01-05T00:01:11.968Z/dlc_2020-01-05T00:01:13.973Z/complete.log
Failed to parse sinopia_export_all_2020-01-05T00:01:11.968Z/minnesota_2020-01-05T00:01:12.425Z/complete.log
Failed to parse sinopia_export_all_2020-01-05T00:01:11.968Z/ld4p_2020-01-05T00:01:12.212Z/complete.log
Failed to parse sinopia_export_all_2020-01-05T00: